In [1]:
import pandas as pd 

In [2]:
inst = pd.read_csv("installments_payments.csv")


In [3]:
inst.head()
 

,SK_ID_PREV,SK_ID_CURR,NUM_INSTALMENT_VERSION,NUM_INSTALMENT_NUMBER,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT
0,1054186,161674,1.0,6,-1180.0,-1187.0,6948.360,6948.360
1,1330831,151639,0.0,34,-2156.0,-2156.0,1716.525,1716.525
2,2085231,193053,2.0,1,-63.0,-63.0,25425.000,25425.000
3,2452527,199697,1.0,3,-2418.0,-2426.0,24350.130,24350.130
4,2714724,167756,1.0,2,-1383.0,-1366.0,2165.040,2160.585


In [4]:
inst.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13605401 entries, 0 to 13605400
Data columns (total 8 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   SK_ID_PREV              int64  
 1   SK_ID_CURR              int64  
 2   NUM_INSTALMENT_VERSION  float64
 3   NUM_INSTALMENT_NUMBER   int64  
 4   DAYS_INSTALMENT         float64
 5   DAYS_ENTRY_PAYMENT      float64
 6   AMT_INSTALMENT          float64
 7   AMT_PAYMENT             float64
dtypes: float64(5), int64(3)
memory usage: 830.4 MB


In [5]:
inst.select_dtypes(include='object').columns
inst.isnull().sum().sort_values(ascending=False).head(20)

DAYS_ENTRY_PAYMENT        2905
AMT_PAYMENT               2905
SK_ID_PREV                   0
SK_ID_CURR                   0
NUM_INSTALMENT_VERSION       0
NUM_INSTALMENT_NUMBER        0
DAYS_INSTALMENT              0
AMT_INSTALMENT               0
dtype: int64

In [6]:
inst = inst.fillna(0)


In [7]:
inst.isnull().sum().sort_values(ascending=False).head(20)

SK_ID_PREV                0
SK_ID_CURR                0
NUM_INSTALMENT_VERSION    0
NUM_INSTALMENT_NUMBER     0
DAYS_INSTALMENT           0
DAYS_ENTRY_PAYMENT        0
AMT_INSTALMENT            0
AMT_PAYMENT               0
dtype: int64

how many payments were late

how many payments were early

how late the worst payment was

total overdue amount

average payment difference

number of missed payments

Create KEY FEATURE: Payment difference

In [8]:
inst["PAY_DIFF"] = inst["AMT_PAYMENT"] - inst["AMT_INSTALMENT"]


Days difference

This tells how early/late the payment was.

In [9]:
inst["DAYS_PAY_DIFF"] = inst["DAYS_ENTRY_PAYMENT"] - inst["DAYS_INSTALMENT"]


Aggregate per loan (SK_ID_PREV)

Because one loan has many payments.

In [10]:
loan_inst_agg = inst.groupby("SK_ID_PREV").agg({
    "PAY_DIFF": ["mean","max","min","sum"],
    "DAYS_PAY_DIFF": ["mean","max","min"],
    "AMT_PAYMENT": ["mean","sum"],
    "AMT_INSTALMENT": ["mean","sum"],
}) 


In [11]:
loan_inst_agg.columns = ["INST_" + "_".join(col).upper() for col in loan_inst_agg.columns]
loan_inst_agg = loan_inst_agg.reset_index()


Merge loan-level installment features into previous applications

In [13]:
import joblib
 
prev = joblib.load("prev_clean.pkl")
 
 

 

In [14]:
prev = prev.merge(loan_inst_agg, on="SK_ID_PREV", how="left")
prev = prev.fillna(0)


Now aggregate previous+installments per customer (SK_ID_CURR)

In [15]:
num_cols = prev.select_dtypes(include=['int64','float64']).columns.tolist()
num_cols.remove("SK_ID_CURR")
num_cols.remove("SK_ID_PREV")


In [16]:
agg_dict = {col: ['mean','max','min','sum'] for col in num_cols}


In [17]:
prev_inst_agg = prev.groupby("SK_ID_CURR").agg(agg_dict)
prev_inst_agg.columns = ["PREV_INST_" + "_".join(col).upper() for col in prev_inst_agg.columns]
prev_inst_agg = prev_inst_agg.reset_index()


In [18]:
import joblib

joblib.dump(prev_inst_agg, "prev_inst_clean.pkl")
joblib.dump(prev, "prev_with_inst_per_loan.pkl")   # optional



['prev_with_inst_per_loan.pkl']